In [1]:
import pandas as pd

In [2]:
DATA = '../data/malcolm_clean.csv'
SAVE_FN = '../data/malcolm_clean_w_duration.csv'
df = pd.read_csv(DATA)

In [3]:
# create columns with earliest day + size
start_day = df.loc[df.groupby('MID')['Day'].idxmin()] 
assert(len(start_day) == df.MID.nunique())
# create columns with latest day + size
end_day = df.loc[df.groupby('MID')['Day'].idxmax()]
assert(len(end_day) == df.MID.nunique())

In [4]:
start_day['Day'].describe()

count    1663.000000
mean       -0.345159
std         0.645208
min        -1.000000
25%        -1.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: Day, dtype: float64

In [5]:
end_day['Day'].describe()

count    1663.000000
mean       24.734215
std         4.628206
min        -1.000000
25%        22.000000
50%        27.000000
75%        28.000000
max        28.000000
Name: Day, dtype: float64

In [6]:
d = df[['Study', 'Group', 'Drug', 'Control', 'MID']].drop_duplicates()
nmids = d.MID.nunique()
assert(len(d) == nmids)
start_day = start_day.rename(columns = {'Day': 'start', 'Size': 'start_size'})
end_day = end_day.rename(columns = {'Day': 'end', 'Size': 'end_size'})
d = d.merge(start_day[['MID', 'start', 'start_size']], on='MID', validate='one_to_one')
assert(len(d) == nmids)
d = d.merge(end_day[['MID', 'end', 'end_size']], on='MID', validate='one_to_one')
assert(len(d) == nmids)
d['duration'] = d['end'] - d['start']
d.head()

,Study,Group,Drug,Control,MID,start,start_size,end,end_size,duration
0,J000100672,3,Cisplatin,0.0,396-11,0.0,212.49,27.0,1275.52,27.0
1,J000100672,3,Cisplatin,0.0,396-20,0.0,180.48,27.0,1115.51,27.0
2,J000100672,3,Cisplatin,0.0,396-21,0.0,76.94,27.0,1056.95,27.0
3,J000100672,3,Cisplatin,0.0,396-31,0.0,151.44,27.0,1330.26,27.0
4,J000100672,3,Cisplatin,0.0,396-32,0.0,127.70,27.0,746.38,27.0


In [7]:
d['duration'].describe()

count    1663.000000
mean       25.079375
std         4.694616
min         0.000000
25%        22.000000
50%        27.000000
75%        28.000000
max        29.000000
Name: duration, dtype: float64

In [9]:
d.MID.nunique()

1663

In [10]:
d.loc[d.duration >= 21].MID.nunique()

1445

In [8]:
d.to_csv(SAVE_FN, index=False)